### Order Matter Origin

In [2]:
from order_matter_ori.block_embedding import generate_block_embedding_cfg_to_file
from order_matter_ori.train_end2end import generate_function_embedding_to_pickle,func_embedding_model_test

RDKit is not installed, which is required for utils related to cheminformatics


In [3]:
bert_epoch = 10
siamese_gnn_epoch = 20

origin_cfg_train_path = "data/train_json/"
origin_cfg_test_path = "data/test_json/"
vocab_path = "data/order_matter/vocab"
bert_model_path = "model_file/order_matter/bert/bert_train_4_task.model"   # bert模型输出文件集基本格式，最终输出模型文件还会在后面加上轮数与损失
block_embedding_train_file_path = "data/order_matter/train_json_block_embedding_graph_4_task/"
block_embedding_test_file_path = "data/order_matter/test_json_block_embedding_graph_4_task/"

func_embedding_model_path = "model_file/order_matter/function_similarity/origin/"
trainset_func_embedding_file_path = "data/order_matter/origin/function_embedding_trainset_epoch{}.pickle".format(19)
testset_func_embedding_file_path = "data/order_matter/origin/function_embedding_testset_epoch{}.pickle".format(19)
func_embedding_test_result = "data/order_matter/origin/"

In [4]:
bert_model_path = "model_file/order_matter/bert/bert_train_4_task.model.ep4"

In [6]:
func_embedding_model_path = "model_file/order_matter/function_similarity/origin/siamese_network_gnn_trained_loss_0.1048.model.ep19"

In [7]:
# 5. 生成函数使用模型向量化的结果
func_embedding_train_file = generate_function_embedding_to_pickle(
    datapath=block_embedding_train_file_path,
    embedding_model_file_path=func_embedding_model_path,
    save_file=trainset_func_embedding_file_path,
)

func_embedding_test_file = generate_function_embedding_to_pickle(
    datapath=block_embedding_test_file_path,
    embedding_model_file_path=func_embedding_model_path,
    save_file=testset_func_embedding_file_path,
)



100%|██████████| 76/76 [07:04<00:00,  5.58s/it]


In [14]:
from util.common_helper import read_pickle,write_pickle
import logging
from util.log_helper import get_logger
from order_matter_ori.function_embedding.similarity_function_find import get_top_k_sim_func,predict_top_k_correct
logger = get_logger('test.log')
def func_embedding_model_test(
        test_file_path,
        result_save_path = "data/order_matter/",
        is_trainset = False
    ):
    logging.info("【Function Embedding Effect Test】")
    if is_trainset:
        logging.info("Trainset:")
    else:
        logging.info("Testset:")

    df_func_data = read_pickle(test_file_path)
    sample_nums = df_func_data.shape[0]

    logging.info("\t测试输入数据：{}".format(test_file_path))
    logging.info("\t查找样本总数:{}".format(sample_nums))
    logging.info("\t源程序包含的函数个数：{}".format(df_func_data.drop_duplicates("func_name").shape[0]))

    embedding_l = df_func_data["func_embedding"].values.tolist()
    func_name_l = df_func_data["func_name"].values.tolist()
    top_5_func,top_5_sim_scores,top_5_funcs_indexs = get_top_k_sim_func(embedding_l,embedding_l,func_name_l,6)
    top_1_func,top_1_sim_scores,top_1_funcs_indexs = get_top_k_sim_func(embedding_l,embedding_l,func_name_l,2)

    df_func_data["top_1_func"] = top_1_func
    df_func_data["top_1_cos_dis"] = top_1_sim_scores
    df_func_data["top_1_func_index"] = top_1_funcs_indexs
    df_func_data["top_5_func"] = top_5_func
    df_func_data["top_5_cos_dis"] = top_5_sim_scores
    df_func_data["top_5_func_index"] = top_5_funcs_indexs
    
    df_func_data["top_1_predict"] = df_func_data.apply(lambda x:predict_top_k_correct(x.func_name,x.top_1_func),axis=1)
    top_1_correct_sample_nums = df_func_data[df_func_data["top_1_predict"]==True].shape[0]
    top_1_acc = top_1_correct_sample_nums/sample_nums
    logging.info("\ttop 1准确率：{}".format(top_1_acc))
    df_func_data["top_5_predict"] = df_func_data.apply(lambda x:predict_top_k_correct(x.func_name,x.top_5_func),axis=1)
    top_5_correct_sample_nums = df_func_data[df_func_data["top_5_predict"]==True].shape[0]
    top_5_acc = top_5_correct_sample_nums/sample_nums
    logging.info("\ttop 5准确率：{}".format(top_5_acc))
    if result_save_path.endswith("/"):
        result_save_path = result_save_path[:-1]

    if is_trainset:
        save_path = "{}/fun_sim_test_{}_{}_{}.pkl".format(result_save_path,"trainset",top_1_acc,top_5_acc)
    else:
        save_path = "{}/fun_sim_test_{}_{}_{}.pkl".format(result_save_path,"testset",top_1_acc,top_5_acc)
    write_pickle(df_func_data,save_path)
    logging.info("\tResult save into:{}".format(save_path))
    return save_path

INFO:root:Log File Save To:     /home/yhk/github/PalmTree/log/test.log
【2022-06-29 11:35:55】INFO: Log File Save To:     /home/yhk/github/PalmTree/log/test.log


In [15]:
# 5. 测试函数编码模型效果
func_embedding_model_test(
    test_file_path= func_embedding_train_file,
    result_save_path = func_embedding_test_result,
    is_trainset=True
)

func_embedding_model_test(
    test_file_path= func_embedding_test_file,
    result_save_path = func_embedding_test_result,
    is_trainset=False
)

INFO:root:【Function Embedding Effect Test】
【2022-06-29 11:35:59】INFO: 【Function Embedding Effect Test】
INFO:root:Trainset:
【2022-06-29 11:35:59】INFO: Trainset:
INFO:root:	测试输入数据：data/order_matter/origin/function_embedding_trainset_epoch19.pickle
【2022-06-29 11:36:02】INFO: 	测试输入数据：data/order_matter/origin/function_embedding_trainset_epoch19.pickle
INFO:root:	查找样本总数:169229
【2022-06-29 11:36:02】INFO: 	查找样本总数:169229
INFO:root:	源程序包含的函数个数：6213
【2022-06-29 11:36:02】INFO: 	源程序包含的函数个数：6213
INFO:root:	top 1准确率：0.8520525441856892
【2022-06-29 11:36:32】INFO: 	top 1准确率：0.8520525441856892
INFO:root:	top 5准确率：0.9084731340373103
【2022-06-29 11:36:40】INFO: 	top 5准确率：0.9084731340373103
INFO:root:	Result save into:data/order_matter/origin/fun_sim_test_trainset_0.8520525441856892_0.9084731340373103.pkl
【2022-06-29 11:36:44】INFO: 	Result save into:data/order_matter/origin/fun_sim_test_trainset_0.8520525441856892_0.9084731340373103.pkl
INFO:root:【Function Embedding Effect Test】
【2022-06-29 11:36:44】INFO: 【F

'data/order_matter/origin/fun_sim_test_testset_0.8358539177013435_0.8939898982547562.pkl'

### Order Matter Atn

In [19]:
from order_matter.block_embedding import generate_block_embedding_cfg_to_file
from order_matter.train_end2end import generate_function_embedding_to_pickle,func_embedding_model_test

In [20]:
bert_epoch = 5
siamese_gnn_epoch = 20

origin_cfg_train_path = "data/train_json/"
origin_cfg_test_path = "data/test_json/"
vocab_path = "data/order_matter/vocab"
bert_model_path = "model_file/order_matter/bert/bert_train_4_task.model"   # bert模型输出文件集基本格式，最终输出模型文件还会在后面加上轮数与损失
block_embedding_train_file_path = "data/order_matter/train_json_block_embedding_graph_4_task/"
block_embedding_test_file_path = "data/order_matter/test_json_block_embedding_graph_4_task/"
func_embedding_model_path = "model_file/order_matter/function_similarity/attention/"
trainset_func_embedding_file_path = "data/order_matter/attention/function_embedding_trainset_epoch{}.pickle".format(19)
testset_func_embedding_file_path = "data/order_matter/attention/function_embedding_testset_epoch{}.pickle".format(19)
func_embedding_test_result = "data/order_matter/attention/"
func_embedding_test_result = "data/order_matter/attention/"

In [21]:
func_embedding_model_path = "model_file/order_matter/function_similarity/attention/siamese_network_gnn_trained_loss_0.06975.model.ep19"

In [22]:
# 5. 生成函数使用模型向量化的结果
func_embedding_train_file = generate_function_embedding_to_pickle(
    datapath=block_embedding_train_file_path,
    embedding_model_file_path=func_embedding_model_path,
    save_file=trainset_func_embedding_file_path,
)

func_embedding_test_file = generate_function_embedding_to_pickle(
    datapath=block_embedding_test_file_path,
    embedding_model_file_path=func_embedding_model_path,
    save_file=testset_func_embedding_file_path,
)

INFO:root:【Generate Function Embedding data to pickle】
【2022-06-29 11:39:21】INFO: 【Generate Function Embedding data to pickle】
INFO:root:	Load function embedding model form model_file/order_matter/function_similarity/attention/siamese_network_gnn_trained_loss_0.06975.model.ep19
【2022-06-29 11:39:21】INFO: 	Load function embedding model form model_file/order_matter/function_similarity/attention/siamese_network_gnn_trained_loss_0.06975.model.ep19
INFO:root:	Load function data from data/order_matter/train_json_block_embedding_graph_4_task/
【2022-06-29 11:39:23】INFO: 	Load function data from data/order_matter/train_json_block_embedding_graph_4_task/
  6%|▌         | 10/177 [02:29<50:40, 18.21s/it]

In [ ]:
# 5. 测试函数编码模型效果
func_embedding_model_test(
    test_file_path= func_embedding_train_file,
    result_save_path = func_embedding_test_result,
    is_trainset=True
)

func_embedding_model_test(
    test_file_path= func_embedding_test_file,
    result_save_path = func_embedding_test_result,
    is_trainset=False
)

NameError: name 'func_embedding_model_test' is not defined